# TensorFlow Recommenders: Quickstart

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

# Import TensorFlow Recommender System (TFRS)

In [1]:
!pip install -q sagemaker==2.9.2
!pip install -q sagemaker-experiments==0.1.24
!pip install -q tensorflow==2.3.0
!pip install -q tensorflow-recommenders==0.2.0
!pip install -q tensorflow-datasets==4.0.0

In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
class RankingModel(tf.keras.Model):

  def __init__(self, embedding_dimension, unique_user_ids, unique_movie_titles):
    super().__init__()

    # Compute embeddings for users.
    self.user_model = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_model = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])
    
  def call(self, inputs):

    user_id, movie_title = inputs

    user_model = self.user_model(user_id)
    movie_model = self.movie_model(movie_title)

    return self.ratings(tf.concat([user_model, movie_model], axis=1))


In [5]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, embedding_dimension, unique_user_ids, unique_movie_titles):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel(embedding_dimension, unique_user_ids, unique_movie_titles)
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    rating_predictions = self.ranking_model(
        (features["user_id"], features["movie_title"]))

    # The task computes the loss and the metrics.
    return self.task(labels=features["user_rating"], predictions=rating_predictions)

# Load Ratings

In [6]:
ratings = tfds.load('movielens/100k-ratings',                     
                    download=False,
                    data_dir='./tensorflow_datasets/',
                    split='train')
print(ratings)

<PrefetchDataset shapes: {bucketized_user_age: (), movie_genres: (None,), movie_id: (), movie_title: (), raw_user_age: (), timestamp: (), user_gender: (), user_id: (), user_occupation_label: (), user_occupation_text: (), user_rating: (), user_zip_code: ()}, types: {bucketized_user_age: tf.float32, movie_genres: tf.int64, movie_id: tf.string, movie_title: tf.string, raw_user_age: tf.float32, timestamp: tf.int64, user_gender: tf.bool, user_id: tf.string, user_occupation_label: tf.int64, user_occupation_text: tf.string, user_rating: tf.float32, user_zip_code: tf.string}>


In [7]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})
print(ratings)

<MapDataset shapes: {movie_title: (), user_id: (), user_rating: ()}, types: {movie_title: tf.string, user_id: tf.string, user_rating: tf.float32}>


# Load Movies

In [8]:
movies = tfds.load('movielens/100k-movies', 
                   download=False,                   
                   data_dir='./tensorflow_datasets/',
                   split='train')
print('Movies BEFORE', movies)

Movies BEFORE <PrefetchDataset shapes: {movie_genres: (None,), movie_id: (), movie_title: ()}, types: {movie_genres: tf.int64, movie_id: tf.string, movie_title: tf.string}>


In [9]:
movies = movies.map(lambda x: x["movie_title"])
print('Movies AFTER', movies)

Movies AFTER <MapDataset shapes: (), types: tf.string>


In [10]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [11]:
movie_titles = ratings.batch(100_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(100_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [13]:
embedding_dimension = 32

In [4]:
# task = tfrs.tasks.Ranking(
#   loss = tf.keras.losses.MeanSquaredError(),
#   metrics=[tf.keras.metrics.RootMeanSquaredError()]
# )

In [14]:
model = MovielensModel(embedding_dimension, unique_user_ids, unique_movie_titles)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [15]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [18]:
epochs = 1

In [19]:
model.fit(cached_train, epochs=epochs)

10/10 [==============================] - 0s 30ms/step - root_mean_squared_error: 1.1091 - loss: 1.2318 - regularization_loss: 0.0000e+00 - total_loss: 1.2318


In [20]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 0s 5ms/step - root_mean_squared_error: 1.1029 - loss: 1.2118 - regularization_loss: 0.0000e+00 - total_loss: 1.2118


{'root_mean_squared_error': 1.102912425994873,
 'loss': 1.1916979551315308,
 'regularization_loss': 0,
 'total_loss': 1.1916979551315308}

# Make Predictions
Use brute-force search to set up retrieval using the trained representations.

In [21]:
index = tfrs.layers.factorized_top_k.BruteForce(model.ranking_model.user_model)
index.index(movies.batch(100).map(model.ranking_model.movie_model), movies)

In [22]:
import os

local_model_dir_bruteforce_model = './exported_models/bruteforce_model/'

tensorflow_saved_model_path_bruteforce_model = os.path.join(local_model_dir_bruteforce_model, 'tensorflow/saved_model/0')

os.makedirs(tensorflow_saved_model_path_bruteforce_model, exist_ok=True)

In [24]:
k = 5
user_id = "42"

_, titles = index(np.array([user_id]))

print(f"Top {k} recommendations for user {user_id}: {titles[0, :k]}")

Top 5 recommendations for user 42: [b'Strawberry and Chocolate (Fresa y chocolate) (1993)'
 b'Murder, My Sweet (1944)'
 b"Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996)"
 b'In the Mouth of Madness (1995)' b'Richie Rich (1994)']


In [25]:
print('Compiled model {}'.format(index))          
print(index.summary())

Compiled model <tensorflow_recommenders.layers.factorized_top_k.BruteForce object at 0x7fc684613990>
Model: "brute_force"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 32)                30208     
Total params: 85,714
Trainable params: 30,208
Non-trainable params: 55,506
_________________________________________________________________
None


In [26]:
index.save(tensorflow_saved_model_path_bruteforce_model, save_format='tf')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: ./exported_models/bruteforce_model/tensorflow/saved_model/0/assets


INFO:tensorflow:Assets written to: ./exported_models/bruteforce_model/tensorflow/saved_model/0/assets


In [27]:
!saved_model_cli show --all --dir $tensorflow_saved_model_path_bruteforce_model

2020-11-03 05:59:04.551923: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-03 05:59:04.551967: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: serving_default_in

In [28]:
user_id = "42"

!saved_model_cli run --input_exprs 'input_1=np.array(["$user_id"])' --tag_set serve --signature_def serving_default --dir ./model/tensorflow/saved_model/0

2020-11-03 05:59:06.817560: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-03 05:59:06.817620: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-11-03 05:59:08.020598: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2020-11-03 05:59:08.020637: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2020-11-03 05:59:08.020670: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395): /proc/driver/nvidia/version does not exist
2020-11-03 05:59:08